# Обучение с подкреплением Q Learning Taxi

**Пространство действий**

Существует 6 дискретных детерминированных действий -двигаться на юг,двигаться на север, двигаться на запад, двигаться на юг , забрать пассажира, высадить пассажира


**Пространство состояний**

Существует 500 дискретных состояний, поскольку имеется 25 позиций такси, 5 возможных местоположений пассажира (включая случай, когда пассажир находится в такси) и 4 местоположения назначения.

Обратите внимание, что существует 400 состояний, которые фактически могут быть достигнуты во время эпизода. Отсутствующие состояния соответствуют ситуациям, в которых пассажир находится в том же месте, что и пункт назначения, поскольку это обычно сигнализирует об окончании эпизода. Четыре дополнительных состояния можно наблюдать сразу после успешных эпизодов, когда и пассажир, и такси находятся в пункте назначения. Это дает в общей сложности 404 достижимых дискретных состояния.

Каждое пространство состояний представлено кортежем: (taxi_row, taxi_col, passenger_location, destination)

Наблюдение - это целое число, которое кодирует соответствующее состояние. Затем кортеж состояний может быть декодирован с помощью метода “decode”.

Места расположения пассажиров:

0. R(ed)

1. G(reen)

2. Y(ellow)

3. B(lue)

4. В такси

Destinations:

0. R(ed)

1. G(reen)

2. Y(ellow)

3. B(lue)

**Награды**

-1 за шаг если не случаются другие награды.

+20 доставка пассажира.

-10 неправильное выполнение действий по “подбору” и “высадке”..

In [1]:
# Virtual display
from pyvirtualdisplay import Display
import numpy as np
import random
import collections
import pickle
import matplotlib.pyplot as plt
import gym
from IPython import display
from tqdm import tqdm
import imageio

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

# Параметры модели

In [2]:
# Тренировочные параметры
n_training_episodes = 10000  # Тренировочных эпизодов
learning_rate = 0.7          # Learning rate

# Параметры тестирования
n_eval_episodes = 10000        # Тестовых эпизодов

# Параметры среды
env_id = "Taxi-v3"     # Имя среды
max_steps = 99               # Максимальное число шагов в эпизоде
gamma = 0.95                 # Discounting rate
eval_seed = []               # The evaluation seed of the environment

# Параметры исследования среды
max_epsilon = 0.8             # Вероятность исследований на старте
min_epsilon = 0.05            # Минимальная вероятность исследований 
decay_rate = 0.0005            # Экспоненциальная скорость затухания для задач исследования

# Функции

In [3]:
def greedy_policy(Qtable, state):
  action = np.argmax(Qtable[state][:])
  
  return action

def epsilon_greedy_policy(Qtable, state, epsilon):
  random_int = random.uniform(0,1)
  if random_int > epsilon:
    action = greedy_policy(Qtable, state)
  else:
    action = env.action_space.sample()
    
  return action

# Создает Qtable размерности (state_space, action_space) и инициализируем  0 
def initialize_q_table(state_space, action_space):
  Qtable = np.zeros((state_space,action_space))
  return Qtable

#Запись видео
def record_video(env, Qtable, out_directory, fps=1):
  """
  Generate a replay video of the agent
  :param env
  :param Qtable: Qtable of our agent
  :param out_directory
  :param fps: how many frame per seconds (with taxi-v3 and frozenlake-v1 we use 1)
  """
  images = []  
  terminated = False
  truncated = False
  state, info = env.reset(seed=random.randint(0,500))
  img = env.render()
  images.append(img)
  while not terminated or truncated:
    action = np.argmax(Qtable[state][:])
    state, reward, terminated, truncated, info = env.step(action) 
    img = env.render()
    images.append(img)
  imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

# Тестирует агента
def evaluate_agent(env, max_steps, n_eval_episodes, Q, seed):
  """
  Evaluate the agent for ``n_eval_episodes`` episodes and returns average reward and std of reward.
  :param env: The evaluation environment
  :param n_eval_episodes: Number of episode to evaluate the agent
  :param Q: The Q-table
  :param seed: The evaluation seed array 
  """
  episode_rewards = []
  for episode in tqdm(range(n_eval_episodes)):
    if seed:
      state, info = env.reset(seed=seed[episode])
    else:
      state, info = env.reset()
    step = 0
    truncated = False
    terminated = False
    total_rewards_ep = 0
    
    for step in range(max_steps):
      
      action = greedy_policy(Q, state)
      new_state, reward, terminated, truncated, info = env.step(action)
      total_rewards_ep += reward
        
      if terminated or truncated:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

# Обучает  агента
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
  for episode in tqdm(range(n_training_episodes)):
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    # Сброс среды
    state, info = env.reset()
    step = 0
    terminated = False
    truncated = False

    # repeat
    for step in range(max_steps):
      # Выбор действия
      action = epsilon_greedy_policy(Qtable, state, epsilon)
      new_state, reward, terminated, truncated, info = env.step(action)

      # Обновление Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
      Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])   

      if terminated or truncated:
        break
      
      state = new_state
  return Qtable

# Инициализация среды

In [4]:
env=gym.make(env_id, render_mode="rgb_array")
n_episodes = 10000
num_actions = env.action_space.n

initial_state = env.reset()
print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")
print(f"Initial state: {initial_state}")

Observation space: Discrete(500)
Action space: Discrete(6)
Initial state: (368, {'prob': 1.0, 'action_mask': array([1, 1, 1, 0, 0, 0], dtype=int8)})


In [5]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")

action_space = env.action_space.n
print("There are ", action_space, " possible actions")

There are  500  possible states
There are  6  possible actions


In [6]:
env.reset()
for _ in range(200):
    env.render()
    env.step(env.action_space.sample()) # take a random action

/home/roman/projects/OTUS_ADV_HW7/venv/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [7]:
# Создаем Qtable размерности (state_space, action_space) и инициализируем  0 
Qtable_taxi = initialize_q_table(state_space, action_space)

# Обучение модели --Q-LEARNING ALGORITHM--

In [8]:
Qtable_taxi = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_taxi)

100%|██████████| 10000/10000 [00:10<00:00, 953.38it/s]


## Тестирование результатов

In [9]:
# Evaluate our Agent
mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_taxi, eval_seed)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

100%|██████████| 10000/10000 [00:03<00:00, 2513.28it/s]

Mean_reward=7.92 +/- 2.58


In [10]:
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

Mean_reward=7.92 +/- 2.58


# Демонстрация модели

In [19]:

record_video(env,Qtable_taxi,'result3.gif')

<img src="result3.gif" width="750" align="center">